In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import json

In [2]:
class Curve:
    # Constructor
    def __init__(self, list_of_points, closed=False):
        self.list_of_points = list_of_points
        self.closed = closed
        self.J = len(list_of_points)
    
    # Square Bracket Overload
    def __getitem__(self, key):
        return self.list_of_points[key % self.J]
    def __setitem__(self, key, value):
        self.list_of_points[key] = value
    
    # Length
    def __len__(self):
        return self.J
    
    # Curve Addition
    def __add__(self, otherCurve):
        if len(self) == len(otherCurve) and self.closed == otherCurve.closed:
            return Curve([self[i] + otherCurve[i] for i in range(len(self))])
    
    # Curve Length
    def curveLength(self):
        l = 0
        for i in range(self.J - 1):
            edgeLength = np.linalg.norm(self[i + 1] - self[i])
            l += edgeLength ** 2
        if self.closed:
            edgeLength = np.linalg.norm(self[-1] - self[0])
            l += edgeLength ** 2
        return l
    

In [ ]:
# Simple Curve Energy Version 1
def curveEnergy(curve):
    J = curve.J

    integral = 0
    for i in range(J):
        for j in range(J):
            # Undefined for i = j
            if i == j:
                continue

            contribution = np.linalg.norm(curve[i] - curve[i+1]) * np.linalg.norm(curve[j] - curve[j+1]) / np.linalg.norm(curve[i] - curve[j])
            integral += contribution
    
    return integral
            
    

In [ ]:

# Simple Curve Energy Version 2
def curveEnergy2(curve):
    J = curve.J

    integral = 0
    for i in range(J):
        for j in range(J):
            # Undefined for |i - j| <= 1
            if abs(i - j) <= 1 or abs(i - j + J) <= 1 or abs(i - j - J) <= 1:
                continue

            contribution = 1 / np.linalg.norm(curve[i] - curve[j]) + 1 / np.linalg.norm(curve[i] - curve[j+1])
            contribution += 1 / np.linalg.norm(curve[i+1] - curve[j]) + 1 / np.linalg.norm(curve[i+1] - curve[j+1])
            contribution /= 4
            integral += contribution * np.linalg.norm(curve[i] - curve[i+1]) * np.linalg.norm(curve[j] - curve[j+1])
    
    return integral

In [3]:
# Tangent-point Curve Energy (One-point Quadrature)
def curveEnergyTangentPointKernelFromMinimal(curve):
    def tangentPointKernelFunction(p, q, T, alpha=2, beta=4.5):
        pmq = p - q
        numerator = np.linalg.norm(np.cross(T, pmq)) ** alpha
        denominator = np.linalg.norm(pmq) ** beta
        return numerator / denominator

    J = curve.J

    integral = 0
    for i in range(J):
        for j in range(J):
            # Undefined for |i - j| <= 1
            if abs(i - j) <= 1 or abs(i - j + J) <= 1 or abs(i - j - J) <= 1:
                continue

            # Checking orientation, then picking two shortest points on each edge i and j
            # Check orientation
            contribution = 0
            xI = curve[i+1] - curve[i]
            if np.dot(curve[i+1]-curve[i], curve[j+1]-curve[j]) > 0:
                # "Same Direction"
                if np.linalg.norm(curve[i]-curve[j]) < np.linalg.norm(curve[i+1]-curve[j+1]):
                    contribution = tangentPointKernelFunction(curve[i], curve[j], xI)
                else:
                    contribution = tangentPointKernelFunction(curve[i+1], curve[j+1], xI)
            else:
                # "Different Direction"
                if np.linalg.norm(curve[i]-curve[j+1]) < np.linalg.norm(curve[i+1]-curve[j]):
                    contribution = tangentPointKernelFunction(curve[i], curve[j+1], xI)
                else:
                    contribution = tangentPointKernelFunction(curve[i+1], curve[j], xI)
            
            integral += contribution * np.linalg.norm(xI) * np.linalg.norm(curve[j] - curve[j+1])
    
    return integral

In [7]:
def curveDifferential(curve, index, perturbation_vec, curveEnergy=curveEnergyTangentPointKernelFromMinimal):
    # Perturb the curve in + and - perturb vec.
    # May be improved.
    curvep = Curve(deepcopy(curve.list_of_points))
    curvep[index] += perturbation_vec
    curven = Curve(deepcopy(curve.list_of_points))
    curven[index] -= perturbation_vec

    energyP = curveEnergy(curvep)
    energyN = curveEnergy(curven)
    differential = (energyP - energyN) / 2
    return differential

In [5]:
def gradientFlow(curve, M=10000, delta_x=0.1, delta_t=0.005, ld=0.0001, skipFrame=10, qPlot=False , xjson="x.json", yjson="y.json", zjson="z.json", qEnergyExport=False, energyjson="energy.json"):
    J = curve.J
    curveN = Curve([np.array([0.0,0.0,0.0]) for i in range(J)])
    xjsonList = []
    yjsonList = []
    zjsonList = []
    energyjsonList = []
    for t in range(M):
        for i in range(J):
            curveN[i][0] = curve[i][0] -curveDifferential(curve, i, np.array([delta_x,0.0,0.0])) / delta_x * delta_t - ld * delta_t * curve[i][0]
            curveN[i][1] = curve[i][1] -curveDifferential(curve, i, np.array([0.0, delta_x, 0])) / delta_x * delta_t - ld * delta_t * curve[i][1]
            curveN[i][2] = curve[i][2] -curveDifferential(curve, i, np.array([0.0, 0.0, delta_x])) / delta_x * delta_t - ld * delta_t * curve[i][2]
        curve = curveN
        xjsonList.append([curve.list_of_points[i][0] for i in range(J)])
        yjsonList.append([curve.list_of_points[i][1] for i in range(J)])
        zjsonList.append([curve.list_of_points[i][2] for i in range(J)])
        if qEnergyExport:
            energyjsonList.append(curveEnergy(curve))

        print(f"Progress: {t}/{M} ({t/M * 100}%)")
        if not qPlot:
            if (t % skipFrame == 0):
                plt.plot([curve.list_of_points[i][0] for i in range(J)], [curve.list_of_points[i][1] for i in range(J)])
                plt.xlim((-1, 1))
                plt.ylim((-1, 1))
                plt.show()
    if qPlot:
        with open(xjson, "w") as file:
            json.dump(xjsonList, file)
        with open(yjson, "w") as file:
            json.dump(yjsonList, file)
        with open(zjson, "w") as file:
            json.dump(zjsonList, file)
    if qEnergyExport:
        with open(energyjson, "w") as file:
            json.dump(energyjsonList, file)
    
            

In [11]:
# Regular Polygon
num_of_verticles = 32
points = []
for i in range(num_of_verticles):
    theta = 2 * np.pi / num_of_verticles * i
    points.append(np.array([np.cos(theta), np.sin(theta), 0]))
curve = Curve(points, True)

In [ ]:
# Fourier Curve
points = []
resolution = 30
x = lambda t: np.cos(t) + 3 * np.cos(2 * t) - np.cos(3 * t) - 0.7 * np.cos(4 * t)
y = lambda t: np.sin(t) + 0.2 * np.cos(2 * t) + 2 * np.sin(3 * t) + 0.2 * np.cos(4 * t)
z = lambda t: np.cos(t) + 2 * np.cos(2 * t) + 4 * np.cos(3 * t) - 2.2 * np.cos(4 * t)
for i in range(resolution):
    theta = 2 * np.pi / resolution * i

    points.append(np.array([x(theta), y(theta), z(theta)]))
fourier = Curve(points)
gradientFlow(fourier, M=10000, delta_x=0.1, delta_t=0.01)

In [13]:
gradientFlow(curve, M=10000, delta_x=0.1, delta_t= 0.005, ld=0.0001, qPlot=True)

Progress: 0/10000 (0.0%)
Progress: 1/10000 (0.01%)
Progress: 2/10000 (0.02%)
Progress: 3/10000 (0.03%)
Progress: 4/10000 (0.04%)
Progress: 5/10000 (0.05%)
Progress: 6/10000 (0.06%)
Progress: 7/10000 (0.06999999999999999%)
Progress: 8/10000 (0.08%)
Progress: 9/10000 (0.09%)
Progress: 10/10000 (0.1%)
Progress: 11/10000 (0.11%)
Progress: 12/10000 (0.12%)
Progress: 13/10000 (0.13%)
Progress: 14/10000 (0.13999999999999999%)
Progress: 15/10000 (0.15%)
Progress: 16/10000 (0.16%)
Progress: 17/10000 (0.16999999999999998%)
Progress: 18/10000 (0.18%)
Progress: 19/10000 (0.19%)
Progress: 20/10000 (0.2%)
Progress: 21/10000 (0.21%)
Progress: 22/10000 (0.22%)
Progress: 23/10000 (0.22999999999999998%)
Progress: 24/10000 (0.24%)
Progress: 25/10000 (0.25%)
Progress: 26/10000 (0.26%)
Progress: 27/10000 (0.27%)
Progress: 28/10000 (0.27999999999999997%)
Progress: 29/10000 (0.29%)
Progress: 30/10000 (0.3%)
Progress: 31/10000 (0.31%)
Progress: 32/10000 (0.32%)
Progress: 33/10000 (0.33%)
Progress: 34/10000 (0

In [ ]:
gradientFlow(curve)